In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/sf-crime/test.csv.zip")

In [ ]:
df.tail()

In [ ]:
train_raw = pd.read_csv("/kaggle/input/sf-crime/train.csv.zip")
test_raw = pd.read_csv("/kaggle/input/sf-crime/test.csv.zip", index_col='Id')
sample_sub = pd.read_csv("/kaggle/input/sf-crime/sampleSubmission.csv.zip")

In [ ]:
test_raw.info()

In [ ]:
train_raw.info()

In [ ]:
print("Length of train data is", len(train_raw))
print("Total Features in the data", len(train_raw.columns)-1)

In [ ]:
print("Total DayOfWeek labels count are", train_raw.DayOfWeek.nunique())
print("Total DayOfWeek labels are", train_raw.DayOfWeek.unique())
plt.figure(figsize=(6, 3))
sns.countplot(x="DayOfWeek", data=train_raw)

In [ ]:
print("Total output labels count are", train_raw.Category.nunique())
print("Total output labels are", train_raw.Category.unique())
plt.figure(figsize=(15, 4))
sns.countplot(x="Category", data=train_raw)

In [ ]:
print("Total PdDistrict labels counts are", train_raw.PdDistrict.nunique())
print("Total PdDistrict labels are", train_raw.PdDistrict.unique())
plt.figure(figsize=(10, 3))
sns.countplot(x="PdDistrict", data=train_raw)

In [ ]:
print("Total Resolution counts labels are", train_raw.Resolution.nunique())
print("Total Resolution labels are", train_raw.Resolution.unique())
plt.figure(figsize=(15, 4))
sns.countplot(x="Resolution", data=train_raw)

In [ ]:
print("Total Descript labels counts are", train_raw.Descript.nunique())
print("Total Address labels counts are", train_raw.Address.nunique())

In [ ]:
y_train = train_raw.Category

In [ ]:
train_cleaned = train_raw.drop(columns=["Address", "Category","Resolution","Descript",  "Dates"])

In [ ]:
test_cleaned = test_raw.drop(columns=["Address", "Dates"])

In [ ]:
train = pd.get_dummies(train_cleaned)

In [ ]:
X_test = pd.get_dummies(test_cleaned)

**Label Encoding**

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_enc = le.fit_transform(y_train)

In [ ]:
y = pd.get_dummies(y_train)

**Scaling**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sclr = MinMaxScaler()
X_train_scaled = sclr.fit_transform(train, y_enc)

In [ ]:
X_test_scaled = sclr.transform(X_test)

**Train-Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_scaled, y_enc, test_size=0.2, random_state=42)

**MODEL**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

***LogisticRegression***

In [ ]:
log_reg = LogisticRegression(verbose=1, solver = 'lbfgs', random_state=42, multi_class='multinomial',).fit(X_train, y_train)

In [ ]:
log_reg_pred = log_reg.score(X_valid, y_valid)
print(log_reg_pred)

***DecisionTreeClassifier***

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_scaled, y, test_size=0.2, random_state=42)

In [ ]:
dtc = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)


In [ ]:
dtc_pred = dtc.score(X_valid, y_valid)
print(dtc_pred)

In [ ]:
sample_sub.columns

In [ ]:
pred = dtc.predict(X_test_scaled)

In [ ]:
out = pd.DataFrame(pred, columns= list(y.columns))

In [ ]:
out['Id'] = X_test.index

In [ ]:
cols = list(out.columns)
cols = [cols[-1]] + cols[:-1]
out = out[cols]

In [ ]:
out.to_csv("dtc_pred_new_1.csv", index=False)